<a href="https://colab.research.google.com/github/SergioManuelJob/ExpertSystemCLIPS/blob/master/CLIPS_Ruta_Mas_Corta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install clipspy

# Importacion de clips
import clips
import logging

Prueba de ClipsPy

In [16]:
# Preparación del entorno
logging.basicConfig(level=logging.DEBUG,
                    force=True,
                    format='CLIPS: %(message)s'
                    )

env = clips.Environment()

TEMPLATE_NODO = """
(deftemplate nodo
   (slot nombre (type SYMBOL))
   (slot vecino (type SYMBOL))
   (slot distancia (type INTEGER))
)"""

TEMPLATE_NODOINICIAL = """
(deftemplate nodoInicial
   (slot nombre)
)
"""

TEMPLATE_RUTA = """
(deftemplate ruta
   (slot inicial (type SYMBOL))
   (slot final (type SYMBOL))
   (slot costo (type INTEGER))
)
"""

DEFFACTS_NODOS = """
(deffacts nodos-iniciales
   (nodo (nombre A) (vecino B) (distancia 10))
   (nodo (nombre A) (vecino C) (distancia 30))
   (nodo (nombre A) (vecino D) (distancia 100))
   (nodo (nombre B) (vecino C) (distancia 5))
   (nodo (nombre C) (vecino D) (distancia 7))
)
"""

DEFRULE_ENCONTRARRUTA= """
(defrule encontrar-ruta
   ?nodoInicial <- (nodoInicial (nombre ?inicio))
   ?nodoVecino <- (nodo (nombre ?inicio) (vecino ?vecino) (distancia ?distancia))
   =>
   (printout t "Calculando la mejor ruta desde el nodo " ?inicio " hacia otros nodos:" crlf)
   (printout t "  " ?vecino " (Costo: " ?distancia ")" crlf)
   (assert (ruta (inicial ?inicio) (final ?vecino) (costo ?distancia)))
)
"""

DEFRULE_ENCONTRARRUTAINDIRECTA = """
(defrule encontrar-rutas-indirectas
   ?ruta <- (ruta (inicial ?inicio) (final ?intermedio) (costo ?costo1))
   ?nodoIntermedio <- (nodo (nombre ?intermedio) (vecino ?final) (distancia ?distancia2))
   (test (not (eq ?intermedio ?inicio)))
   =>
   (bind ?costoTotal (+ ?costo1 ?distancia2))
   (printout t "  " ?final " (Costo: " ?costoTotal ")" crlf)
   (assert (ruta (inicial ?inicio) (final ?final) (costo ?costoTotal)))
   (retract ?ruta)
)
"""

# Falta el reset y el assert

env.build(TEMPLATE_NODO)
env.build(TEMPLATE_NODOINICIAL)
env.build(TEMPLATE_RUTA)
env.build(DEFFACTS_NODOS)
env.reset()

template = env.find_template('nodoInicial')

fact = template.assert_fact(nombre='A')
assert fact['nombre'] == 'A'

env.build(DEFRULE_ENCONTRARRUTA)
env.build(DEFRULE_ENCONTRARRUTAINDIRECTA)

router = clips.LoggingRouter()
env.add_router(router)
env.run()

0

In [17]:
facts = env.facts()

# Imprimir los hechos
print("Hechos después de la ejecución:")
for fact in facts:
    print(fact)

Hechos después de la ejecución:
(nodo (nombre A) (vecino B) (distancia 10))
(nodo (nombre A) (vecino C) (distancia 30))
(nodo (nombre A) (vecino D) (distancia 100))
(nodo (nombre B) (vecino C) (distancia 5))
(nodo (nombre C) (vecino D) (distancia 7))
(nodoInicial (nombre "A"))
